# Battery Fast Charging Optimization using Neural Networks

This notebook demonstrates the optimization of battery fast charging parameters using artificial neural networks (ANNs). The goal is to optimize anode and cathode porosity to maximize battery capacity.

## Overview
1. **Data Loading**: Load training data for anode/cathode porosity and corresponding capacity values
2. **Data Preprocessing**: Normalize input features and target values
3. **Neural Network Training**: Train an ANN to predict battery capacity from porosity parameters
4. **Optimization**: Use gradient-based optimization to find optimal porosity values
5. **Results Visualization**: Plot optimization progress and calculate final optimized values

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## 1. Import Required Libraries

Import necessary libraries for neural network implementation, data processing, and visualization:
- **PyTorch**: For neural network creation and training
- **NumPy**: For numerical operations and data handling
- **Matplotlib**: For plotting results and optimization progress

In [ ]:
input_data_raw1 = np.loadtxt('input_data_save.txt')
cap_data_raw1 = np.loadtxt('cap_data_save.txt')

## 2. Load Training Data

Load the pre-generated training dataset:
- **input_data_save.txt**: Contains anode and cathode porosity values (2 features)
- **cap_data_save.txt**: Contains corresponding battery capacity values (target variable)

The training data represents the relationship between electrode porosity and battery performance.

In [ ]:
# Convert the training data to PyTorch tensors
x_train_tensor_raw = torch.tensor(input_data_raw1, dtype=torch.float32)
y_train_tensor_raw = torch.tensor(cap_data_raw1, dtype=torch.float32)

## 3. Convert Data to PyTorch Tensors

Convert the loaded NumPy arrays to PyTorch tensors for compatibility with neural network operations:
- Input features (x_train_tensor_raw): Porosity values for anode and cathode
- Target values (y_train_tensor_raw): Corresponding battery capacity measurements

In [ ]:
def normalize_tensor(tensor, max_val, min_val):
    return (tensor - min_val) / (max_val - min_val)

x_train_epsneg_raw1 = x_train_tensor_raw[:, 0]
x_train_epspos_raw1 = x_train_tensor_raw[:, 1]

max_epsneg = torch.max(x_train_epsneg_raw1)
min_epsneg = torch.min(x_train_epsneg_raw1)

max_epspos = torch.max(x_train_epspos_raw1)
min_epspos = torch.min(x_train_epspos_raw1)

x_train_epsneg_norm1 = normalize_tensor(x_train_epsneg_raw1, max_epsneg, min_epsneg)
x_train_epspos_norm1 = normalize_tensor(x_train_epspos_raw1, max_epspos, min_epspos)
x_train = torch.stack((x_train_epsneg_norm1, x_train_epspos_norm1), dim=1)

y_train_tensor = y_train_tensor_raw
max_y = torch.max(y_train_tensor)
min_y = torch.min(y_train_tensor)

y_train_norm = normalize_tensor(y_train_tensor, max_y, min_y)
y_train = y_train_norm.reshape(200,1)

## 4. Data Normalization

Normalize the input features and target values to improve neural network training performance:

1. **Feature Normalization**: Scale anode and cathode porosity values to [0,1] range
2. **Target Normalization**: Scale capacity values to [0,1] range
3. **Store Min/Max Values**: Keep original bounds for later denormalization

Normalization helps with:
- Faster convergence during training
- Better gradient flow
- Numerical stability

In [ ]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        # Define the layers
        self.layer1 = nn.Linear(2, 15)  # Input layer (2 inputs) to first hidden layer (15 neurons)
        self.layer2 = nn.Linear(15, 5)  # First hidden layer (15 neurons) to second hidden layer (5 neurons)
        self.layer3 = nn.Linear(5, 1)   # Second hidden layer (5 neurons) to output layer (1 output)
        
        # Define activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Forward pass through the network
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Instantiate the model
model = ANN()

# Define the loss function (Mean Squared Error for regression)
criterion = nn.MSELoss()

# Define the optimizer (Stochastic Gradient Descent)
optimizer = optim.SGD(model.parameters(), lr=0.05)

## 5. Define Neural Network Architecture

Create an Artificial Neural Network (ANN) to learn the relationship between porosity and battery capacity:

**Network Architecture:**
- **Input Layer**: 2 neurons (anode porosity, cathode porosity)
- **Hidden Layer 1**: 15 neurons with ReLU activation
- **Hidden Layer 2**: 5 neurons with ReLU activation  
- **Output Layer**: 1 neuron (battery capacity)

**Training Setup:**
- **Loss Function**: Mean Squared Error (MSE) for regression
- **Optimizer**: Stochastic Gradient Descent (SGD) with learning rate 0.05

In [ ]:
# Number of epochs for training
num_epochs = 20000

for epoch in range(num_epochs):
    # Forward pass: compute the model output
    outputs = model(x_train)
    
    # Compute the loss
    loss = criterion(outputs, y_train)
    
    # Backward pass: compute the gradients
    optimizer.zero_grad()
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    # Print the loss every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

## 6. Train the Neural Network

Train the ANN for 20,000 epochs to learn the porosity-capacity relationship:

**Training Process:**
1. **Forward Pass**: Compute predicted capacity from input porosity values
2. **Loss Calculation**: Compare predictions with actual capacity values using MSE
3. **Backward Pass**: Calculate gradients using backpropagation
4. **Parameter Update**: Adjust weights using SGD optimizer

The training loss is printed every 100 epochs to monitor convergence.

In [ ]:
# Define the function to optimize
def objective_function(x):
    tensor1 = x[0]    # 1 represents the anode porosity 
    tensor2 = x[1]    # 2 represents the cathode porosity
    x_input = torch.stack((tensor1, tensor2), dim=0)
    return model(x_input)

# Initialize the inputs (parameters to optimize)
val0 = torch.rand(1)    # an initial guess of anode porosity (normalized)
val1 = torch.rand(1)    # an initial guess of cathode porosity (normalized)
x = torch.tensor([val0, val1], requires_grad=True)

# Define the optimizer (Stochastic Gradient Descent)
optimizer = torch.optim.SGD([x], lr=0.01)  # lr is the learning rate

# Number of optimization steps
num_steps = 100

# Lists to store the evolution of x[0], x[1], and loss
x0_values = []
x1_values = []

# Optimization loop
for step in range(num_steps):
    # Zero the gradients from the previous step
    optimizer.zero_grad()

    # Compute the objective function
    result = objective_function(x)
    loss = -result[0]

    # Compute the gradients (backward pass)
    loss.backward()

    # Update the parameters using the optimizer
    optimizer.step()

    # Apply the constraints to keep x[0] and x[1] within [0, 1]
    with torch.no_grad():
        x.clamp_(0, 1)  # In-place operation to clamp x within [0, 1]

    # Store the current values of x[0], x[1], and loss
    x0_values.append(x[0].item())
    x1_values.append(x[1].item())

    # Print the current value of the loss and parameters
    print(f"Step {step + 1}: x = {x.detach().numpy()}, Loss = {loss.item()}")

# Final optimized values
print(f"Optimized x: {x.detach().numpy()}")
print(f"Final Loss: {objective_function(x).item()}")

## 7. Optimize Battery Parameters

Use the trained neural network to find optimal porosity values that maximize battery capacity:

**Optimization Strategy:**
1. **Objective Function**: Use the trained ANN to predict capacity from porosity values
2. **Gradient-Based Optimization**: Maximize capacity by minimizing negative capacity
3. **Constraints**: Keep porosity values within feasible bounds [0,1] using clamping
4. **Optimizer**: SGD with learning rate 0.01 for 100 optimization steps

The optimization tracks the evolution of both anode and cathode porosity values.

In [ ]:
# Plotting the evolution of x[0] and x[1]
plt.figure(figsize=(8, 6))
plt.plot(x0_values, label='x[0]')
plt.plot(x1_values, label='x[1]')
plt.xlabel('Step')
plt.ylabel('Value')
plt.title('Evolution of x[0] and x[1]')
plt.legend()
plt.show()

## 8. Visualize Optimization Progress

Plot the evolution of porosity parameters during the optimization process:
- **x[0]**: Anode porosity evolution over optimization steps
- **x[1]**: Cathode porosity evolution over optimization steps

This visualization helps understand how the optimizer converges to the optimal values.

In [ ]:
# define the de-normalization function
def de_normalize_tensor(tensor, max_val, min_val):
    return tensor * (max_val - min_val) + min_val
    
# de-normalize the porosity of anode and cathode
value_real0 = de_normalize_tensor(x[0], max_epsneg, min_epsneg)
print(value_real0)    # optimized result of anode porosity
value_real1 = de_normalize_tensor(x[1], max_epspos, min_epspos)
print(value_real1)    # optimized result of cathode porosity

# de-normalize the capacity value (C/m^2)
y = model(x)
cap_optim = de_normalize_tensor(y[0], max_y, min_y)
print(cap_optim)

## 9. Extract Final Optimized Results

Convert the normalized optimization results back to their original scales:

1. **Denormalize Porosity Values**: Convert optimal anode and cathode porosity from [0,1] range back to physical units
2. **Calculate Optimal Capacity**: Use the trained model to predict the maximum achievable capacity
3. **Display Results**: Show the final optimized porosity values and corresponding capacity

This gives us the practical battery design parameters for maximum fast charging performance.